<a href="https://colab.research.google.com/github/avionerman/machine_learning_2025/blob/main/poverty_competition_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import zipfile
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.optimize import minimize

import xgboost as xgb
import lightgbm as lgb

try:
    from catboost import CatBoostRegressor
except ImportError:
    print("CatBoost not found, installing...")
    !pip install catboost -qq
    from catboost import CatBoostRegressor

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [43]:

class Config:
    """Central configuration for all parameters."""

    TRAIN_FEATURES_PATH = '/content/train_hh_features.csv'
    TRAIN_LABELS_PATH = '/content/train_hh_gt.csv'
    TEST_FEATURES_PATH = '/content/test_hh_features.csv'

    # target col
    TARGET_COL = 'cons_ppp17'

    # ID cols
    ID_COLS = ['survey_id', 'hhid']

    RANDOM_STATE = 42

    # validation split
    VAL_SIZE = 0.2

    # poverty thresholds for competition metric
    POVERTY_THRESHOLDS = [
        3.17, 3.94, 4.60, 5.26, 5.88, 6.47, 7.06, 7.70,
        8.40, 9.13, 9.87, 10.70, 11.62, 12.69, 14.03,
        15.64, 17.76, 20.99, 27.37
    ]

    # percentile ranks for weighting
    PERCENTILE_RANKS = [
        0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40,
        0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80,
        0.85, 0.90, 0.95
    ]

    # survey IDs
    TEST_SURVEY_IDS = [400000, 500000, 600000]

    # binary mappings
    BINARY_MAPPINGS = {
        'Yes': 1, 'No': 0,
        'Male': 1, 'Female': 0,
        'Access': 1, 'No access': 0,
        'Urban': 1, 'Rural': 0,
        'Owner': 1, 'Renter': 0, 'Not owner': 0,
        'Employed': 1, 'Not employed': 0,
        1: 1, 0: 0,
        1.0: 1, 0.0: 0
    }



In [44]:

class DataProcessor:
    """Handles all data loading, cleaning, and preprocessing."""

    def __init__(self, config):
        self.config = config
        self.label_encoders = {}
        self.scaler = StandardScaler()
        self.feature_cols = None

    def load_data(self):
        """Load training and test data."""
        print("Loading data...")

        train_features = pd.read_csv(self.config.TRAIN_FEATURES_PATH)
        train_labels = pd.read_csv(self.config.TRAIN_LABELS_PATH)
        test_features = pd.read_csv(self.config.TEST_FEATURES_PATH)

        train = train_features.merge(train_labels, on=self.config.ID_COLS)

        print(f"Training data shape: {train.shape}")
        print(f"Test data shape: {test_features.shape}")

        return train, test_features

    def handle_missing_values(self, train, test):
        """Fill missing values in both datasets."""
        print("Handling missing values...")

        train['sector1d'] = train['sector1d'].fillna('Not employed')
        test['sector1d'] = test['sector1d'].fillna('Not employed')

        mode_dweltyp = train['dweltyp'].mode()[0]
        train['dweltyp'] = train['dweltyp'].fillna(mode_dweltyp)
        test['dweltyp'] = test['dweltyp'].fillna(mode_dweltyp)

        median_utl = train['utl_exp_ppp17'].median()
        train['utl_exp_ppp17'] = train['utl_exp_ppp17'].fillna(median_utl)
        test['utl_exp_ppp17'] = test['utl_exp_ppp17'].fillna(median_utl)

        cols_to_fill = ['employed', 'share_secondary', 'educ_max']
        for col in cols_to_fill:
            if col in train.columns and train[col].isnull().sum() > 0:
                mode_val = train[col].mode()[0]
                train[col] = train[col].fillna(mode_val)
                test[col] = test[col].fillna(mode_val)

        consumed_cols = [col for col in train.columns if col.startswith('consumed')]
        for col in consumed_cols:
            if train[col].isnull().sum() > 0:
                mode_val = train[col].mode()[0]
                train[col] = train[col].fillna(mode_val)
                test[col] = test[col].fillna(mode_val)

        print(f"Missing values after filling: {train.isnull().sum().sum()}")
        return train, test

    def encode_binary_columns(self, train, test):
        """Encode binary columns with robust handling."""
        print("Encoding binary columns...")

        consumed_cols = [col for col in train.columns if col.startswith('consumed')]
        binary_cols = ['male', 'owner', 'water', 'toilet', 'sewer', 'elect',
                       'employed', 'any_nonagric', 'urban'] + consumed_cols

        for col in binary_cols:
            if col in train.columns:
                unique_train = train[col].unique()
                unique_test = test[col].unique()

                if train[col].dtype in ['int64', 'float64', 'int32', 'float32']:
                    train[col] = train[col].astype(int)
                    test[col] = test[col].astype(int)
                else:
                    train[col] = train[col].map(self.config.BINARY_MAPPINGS)
                    test[col] = test[col].map(self.config.BINARY_MAPPINGS)

                    if train[col].isnull().any():
                        print(f"  Warning: Unmapped values in {col} (train). Unique before: {unique_train}")
                        train[col] = train[col].fillna(0)
                    if test[col].isnull().any():
                        print(f"  Warning: Unmapped values in {col} (test). Unique before: {unique_test}")
                        test[col] = test[col].fillna(0)

                    train[col] = train[col].astype(int)
                    test[col] = test[col].astype(int)

        print(f"Binary columns encoded: {len(binary_cols)}")
        return train, test, binary_cols

    def encode_multiclass_columns(self, train, test):
        """Encode multiclass columns using LabelEncoder."""
        print("Encoding multiclass columns...")

        multiclass_cols = ['water_source', 'sanitation_source', 'dweltyp', 'educ_max', 'sector1d']

        for col in multiclass_cols:
            if col in train.columns:
                le = LabelEncoder()
                train[col] = train[col].astype(str)
                test[col] = test[col].astype(str)

                combined = pd.concat([train[col], test[col]], axis=0)
                le.fit(combined)
                train[col] = le.transform(train[col])
                test[col] = le.transform(test[col])
                self.label_encoders[col] = le

        print(f"Multiclass columns encoded: {len(multiclass_cols)}")
        return train, test, multiclass_cols

    def define_feature_columns(self, train, binary_cols, multiclass_cols):
        """Define all feature columns."""
        print("Defining feature columns...")

        numerical_cols = [
            'weight', 'utl_exp_ppp17', 'hsize', 'num_children5', 'num_children10',
            'num_children18', 'age', 'num_adult_female', 'num_adult_male',
            'num_elderly', 'sworkershh', 'share_secondary', 'sfworkershh'
        ]

        region_cols = ['region1', 'region2', 'region3', 'region4', 'region5', 'region6', 'region7']

        feature_cols = binary_cols + multiclass_cols + numerical_cols + region_cols
        feature_cols = [col for col in feature_cols if col in train.columns and col != self.config.TARGET_COL]
        feature_cols = list(dict.fromkeys(feature_cols))

        self.feature_cols = feature_cols
        print(f"Total feature columns: {len(feature_cols)}")

        return feature_cols

    def prepare_data(self, train, test, feature_cols):
        """Prepare feature matrices and target."""
        print("Preparing feature matrices...")

        X = train[feature_cols].copy()
        y = train[self.config.TARGET_COL].copy()
        X_test = test[feature_cols].copy()

        sample_weights = train['weight'].values
        survey_ids = train['survey_id'].values

        y_log = np.log1p(y)

        print(f"X shape: {X.shape}")
        print(f"y shape: {y.shape}")
        print(f"X_test shape: {X_test.shape}")

        return X, y_log, X_test, sample_weights, survey_ids

    def create_splits(self, X, y_log, sample_weights, survey_ids):
        """Create train/validation splits with survey IDs."""
        print("Creating train/validation splits...")

        x_train, x_val, y_train, y_val, w_train, w_val, sid_train, sid_val = train_test_split(
            X, y_log, sample_weights, survey_ids,
            test_size=self.config.VAL_SIZE,
            random_state=self.config.RANDOM_STATE
        )

        print(f"Training set: {x_train.shape[0]} samples")
        print(f"Validation set: {x_val.shape[0]} samples")
        print(f"Surveys in validation: {np.unique(sid_val)}")

        return x_train, x_val, y_train, y_val, w_train, w_val, sid_train, sid_val

    def scale_features(self, x_train, x_val, X, X_test):
        """Scale features for neural network."""
        print("Scaling features...")

        x_train_scaled = self.scaler.fit_transform(x_train)
        x_val_scaled = self.scaler.transform(x_val)
        X_scaled = self.scaler.fit_transform(X)
        X_test_scaled = self.scaler.transform(X_test)

        return x_train_scaled, x_val_scaled, X_scaled, X_test_scaled

    def process_all(self):
        """Run the complete data processing pipeline."""
        print("=" * 60)
        print("STARTING DATA PROCESSING PIPELINE")
        print("=" * 60)

        train, test = self.load_data()
        train, test = self.handle_missing_values(train, test)
        train, test, binary_cols = self.encode_binary_columns(train, test)
        train, test, multiclass_cols = self.encode_multiclass_columns(train, test)
        feature_cols = self.define_feature_columns(train, binary_cols, multiclass_cols)
        X, y_log, X_test, sample_weights, survey_ids = self.prepare_data(train, test, feature_cols)
        x_train, x_val, y_train, y_val, w_train, w_val, sid_train, sid_val = self.create_splits(
            X, y_log, sample_weights, survey_ids
        )
        x_train_scaled, x_val_scaled, X_scaled, X_test_scaled = self.scale_features(x_train, x_val, X, X_test)

        print("\n" + "=" * 60)
        print("DATA PROCESSING COMPLETE")
        print("=" * 60)

        return {
            'train': train,
            'test': test,
            'X': X,
            'y_log': y_log,
            'X_test': X_test,
            'sample_weights': sample_weights,
            'survey_ids': survey_ids,
            'x_train': x_train,
            'x_val': x_val,
            'y_train': y_train,
            'y_val': y_val,
            'w_train': w_train,
            'w_val': w_val,
            'sid_train': sid_train,
            'sid_val': sid_val,
            'x_train_scaled': x_train_scaled,
            'x_val_scaled': x_val_scaled,
            'X_scaled': X_scaled,
            'X_test_scaled': X_test_scaled,
            'feature_cols': feature_cols
        }


In [45]:

class CompetitionMetric:
    """
    Calculates the competition metric exactly as defined:
    metric = (1/S) × Σs [ (90/Σw) × Σt(wt × |rt - r̂t|/rt) + (10/H) × Σh(|ch - ĉh|/ch) ]

    Key: Calculate per survey, then average across surveys.
    """

    def __init__(self, config):
        self.config = config
        self.threshold_weights = [1 - abs(0.4 - p) for p in config.PERCENTILE_RANKS]
        self.sum_weights = sum(self.threshold_weights)

    def calculate(self, y_pred_log, y_true_log, survey_ids=None, return_details=False):
        """
        Calculate competition metric for predictions in log scale.
        """
        y_pred = np.expm1(y_pred_log)
        y_true = np.expm1(y_true_log)

        return self.calculate_from_original_scale(y_pred, y_true, survey_ids, return_details)

    def calculate_from_original_scale(self, y_pred, y_true, survey_ids=None, return_details=False):
        """
        Calculate competition metric when predictions are already in original scale.
        """
        if survey_ids is None:
            survey_ids = np.zeros(len(y_true), dtype=int)

        unique_surveys = np.unique(survey_ids)

        survey_scores = []
        survey_poverty_mapes = []
        survey_consumption_mapes = []

        for survey_id in unique_surveys:
            mask = survey_ids == survey_id
            y_true_survey = y_true[mask]
            y_pred_survey = y_pred[mask]

            poverty_errors = []
            for threshold, tw in zip(self.config.POVERTY_THRESHOLDS, self.threshold_weights):
                actual_rate = (y_true_survey < threshold).mean()
                pred_rate = (y_pred_survey < threshold).mean()
                if actual_rate > 0:
                    error = tw * abs(actual_rate - pred_rate) / actual_rate
                    poverty_errors.append(error)

            poverty_mape = sum(poverty_errors) / self.sum_weights if poverty_errors else 0
            consumption_mape = np.mean(np.abs(y_true_survey - y_pred_survey) / y_true_survey)
            survey_score = 0.9 * poverty_mape + 0.1 * consumption_mape

            survey_scores.append(survey_score)
            survey_poverty_mapes.append(poverty_mape)
            survey_consumption_mapes.append(consumption_mape)

        final_score = np.mean(survey_scores)
        avg_poverty_mape = np.mean(survey_poverty_mapes)
        avg_consumption_mape = np.mean(survey_consumption_mapes)

        if return_details:
            return final_score, avg_poverty_mape, avg_consumption_mape
        return final_score



In [46]:

class ModelTrainer:
    """Trains machine learning models: XGBoost, LightGBM, CatBoost, Neural Network."""

    def __init__(self, config, metric):
        self.config = config
        self.metric = metric
        self.models = {}
        self.val_predictions = {}
        self.test_predictions = {}

    def train_xgboost(self, X, y_log, x_val, y_val, sid_val):
        """Train XGBoost model."""
        print("\n" + "=" * 50)
        print("Training XGBoost...")
        print("=" * 50)

        model = xgb.XGBRegressor(
            n_estimators=300,
            max_depth=8,
            learning_rate=0.05,
            subsample=0.7,
            colsample_bytree=0.8,
            min_child_weight=3,
            random_state=self.config.RANDOM_STATE,
            tree_method='hist'
        )

        start_time = time.time()
        model.fit(X, y_log)
        train_time = time.time() - start_time

        print(f"Training time: {train_time:.2f} seconds")

        y_pred_val = model.predict(x_val)
        score, pov_mape, cons_mape = self.metric.calculate(
            y_pred_val, y_val.values, sid_val, return_details=True
        )
        print(f"Validation Score: {score*100:.2f}% (Poverty: {pov_mape*100:.2f}%, Consumption: {cons_mape*100:.2f}%)")

        self.models['XGBoost'] = model
        self.val_predictions['XGBoost'] = y_pred_val

        return model

    def train_lightgbm(self, X, y_log, x_val, y_val, sid_val):
        """Train LightGBM model."""
        print("\n" + "=" * 50)
        print("Training LightGBM...")
        print("=" * 50)

        model = lgb.LGBMRegressor(
            n_estimators=300,
            max_depth=8,
            learning_rate=0.05,
            subsample=0.7,
            colsample_bytree=0.8,
            random_state=self.config.RANDOM_STATE,
            verbose=-1
        )

        start_time = time.time()
        model.fit(X, y_log)
        train_time = time.time() - start_time

        print(f"Training time: {train_time:.2f} seconds")

        y_pred_val = model.predict(x_val)
        score, pov_mape, cons_mape = self.metric.calculate(
            y_pred_val, y_val.values, sid_val, return_details=True
        )
        print(f"Validation Score: {score*100:.2f}% (Poverty: {pov_mape*100:.2f}%, Consumption: {cons_mape*100:.2f}%)")

        self.models['LightGBM'] = model
        self.val_predictions['LightGBM'] = y_pred_val

        return model

    def train_catboost(self, X, y_log, x_val, y_val, sid_val):
        """Train CatBoost model on GPU."""
        print("\n" + "=" * 50)
        print("Training CatBoost (GPU)...")
        print("=" * 50)

        model = CatBoostRegressor(
            iterations=500,
            depth=8,
            learning_rate=0.05,
            random_seed=self.config.RANDOM_STATE,
            verbose=50,
            task_type='GPU'
        )

        start_time = time.time()
        model.fit(X, y_log)
        train_time = time.time() - start_time

        print(f"Training time: {train_time:.2f} seconds")

        y_pred_val = model.predict(x_val)
        score, pov_mape, cons_mape = self.metric.calculate(
            y_pred_val, y_val.values, sid_val, return_details=True
        )
        print(f"Validation Score: {score*100:.2f}% (Poverty: {pov_mape*100:.2f}%, Consumption: {cons_mape*100:.2f}%)")

        self.models['CatBoost'] = model
        self.val_predictions['CatBoost'] = y_pred_val

        return model

    def train_neural_network(self, X_scaled, y_log, x_val_scaled, y_val, sid_val):
        """Train Neural Network model."""
        print("\n" + "=" * 50)
        print("Training Neural Network...")
        print("=" * 50)

        model = keras.Sequential([
            layers.Dense(128, activation='relu', input_shape=(X_scaled.shape[1],)),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            layers.Dense(64, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            layers.Dense(32, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.2),
            layers.Dense(1)
        ])

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss='mse',
            metrics=['mae']
        )

        early_stop = keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )

        start_time = time.time()
        model.fit(
            X_scaled, y_log,
            validation_split=0.2,
            epochs=100,
            batch_size=256,
            callbacks=[early_stop],
            verbose=1
        )
        train_time = time.time() - start_time

        print(f"Training time: {train_time:.2f} seconds")

        y_pred_val = model.predict(x_val_scaled, verbose=0).flatten()
        score, pov_mape, cons_mape = self.metric.calculate(
            y_pred_val, y_val.values, sid_val, return_details=True
        )
        print(f"Validation Score: {score*100:.2f}% (Poverty: {pov_mape*100:.2f}%, Consumption: {cons_mape*100:.2f}%)")

        self.models['NeuralNet'] = model
        self.val_predictions['NeuralNet'] = y_pred_val

        return model

    def train_all(self, data):
        """Train all models."""
        print("\n" + "=" * 60)
        print("TRAINING ALL MODELS")
        print("=" * 60)

        self.train_xgboost(
            data['X'], data['y_log'],
            data['x_val'], data['y_val'], data['sid_val']
        )

        self.train_lightgbm(
            data['X'], data['y_log'],
            data['x_val'], data['y_val'], data['sid_val']
        )

        self.train_catboost(
            data['X'], data['y_log'],
            data['x_val'], data['y_val'], data['sid_val']
        )

        self.train_neural_network(
            data['X_scaled'], data['y_log'],
            data['x_val_scaled'], data['y_val'], data['sid_val']
        )

        print("\n" + "=" * 60)
        print("ALL MODELS TRAINED")
        print("=" * 60)

        return self.models, self.val_predictions

    def generate_test_predictions(self, X_test, X_test_scaled):
        """Generate predictions on test set for all models."""
        print("\nGenerating test predictions for all models...")

        self.test_predictions['XGBoost'] = self.models['XGBoost'].predict(X_test)
        self.test_predictions['LightGBM'] = self.models['LightGBM'].predict(X_test)
        self.test_predictions['CatBoost'] = self.models['CatBoost'].predict(X_test)
        self.test_predictions['NeuralNet'] = self.models['NeuralNet'].predict(X_test_scaled, verbose=0).flatten()

        print("Test predictions generated for all models")

        return self.test_predictions


In [47]:

class EnsembleOptimizer:
    """Optimizes blend weights for competition metric."""

    def __init__(self, config, metric):
        self.config = config
        self.metric = metric
        self.optimal_weights = None
        self.model_names = None
        self.best_single_model = None
        self.best_single_score = None

    def optimize_weights(self, val_predictions, y_val, sid_val):
        """Find optimal blend weights using competition metric."""
        print("\n" + "=" * 60)
        print("OPTIMIZING BLEND WEIGHTS FOR COMPETITION METRIC")
        print("=" * 60)

        self.model_names = list(val_predictions.keys())
        predictions_list = [val_predictions[name] for name in self.model_names]

        print("\nIndividual Model Scores (per survey metric):")
        print("-" * 40)
        self.best_single_score = float('inf')
        best_single_idx = 0

        for i, (name, pred) in enumerate(zip(self.model_names, predictions_list)):
            score = self.metric.calculate(pred, y_val.values, sid_val)
            print(f"  {name}: {score*100:.2f}%")
            if score < self.best_single_score:
                self.best_single_score = score
                best_single_idx = i
                self.best_single_model = name

        print(f"\nBest single model: {self.best_single_model} ({self.best_single_score*100:.2f}%)")

        def objective(w):
            weights = np.array(w) / np.sum(w)
            blended = np.zeros_like(predictions_list[0])
            for weight, pred in zip(weights, predictions_list):
                blended += weight * pred
            return self.metric.calculate(blended, y_val.values, sid_val)

        n_models = len(predictions_list)
        best_result = None
        best_score = float('inf')

        starting_points = [
            np.ones(n_models) / n_models,
            np.eye(n_models)[best_single_idx],
            np.array([0.6, 0.2, 0.1, 0.1][:n_models]),
            np.array([0.4, 0.3, 0.2, 0.1][:n_models]),
            np.array([0.5, 0.3, 0.2, 0.0][:n_models]),
        ]

        for start in starting_points:
            if len(start) != n_models:
                start = np.ones(n_models) / n_models

            result = minimize(
                objective,
                start,
                method='SLSQP',
                bounds=[(0, 1)] * n_models,
                options={'maxiter': 1000}
            )

            if result.fun < best_score:
                best_score = result.fun
                best_result = result

        self.optimal_weights = best_result.x / best_result.x.sum()

        print("\nOPTIMAL BLEND WEIGHTS:")
        print("-" * 40)
        for name, w in zip(self.model_names, self.optimal_weights):
            if w > 0.01:
                print(f"  {name}: {w*100:.1f}%")

        blended_pred = np.zeros_like(predictions_list[0])
        for name, w in zip(self.model_names, self.optimal_weights):
            blended_pred += w * val_predictions[name]

        blended_score, pov_mape, cons_mape = self.metric.calculate(
            blended_pred, y_val.values, sid_val, return_details=True
        )

        print(f"\nBLENDED VALIDATION SCORE: {blended_score*100:.2f}%")
        print(f"  Poverty MAPE:    {pov_mape*100:.2f}%")
        print(f"  Consumption MAPE: {cons_mape*100:.2f}%")

        if blended_score > self.best_single_score:
            print(f"\n WARNING: Blend ({blended_score*100:.2f}%) is worse than {self.best_single_model} ({self.best_single_score*100:.2f}%)")
            print(f"    Using {self.best_single_model} alone instead!")
            self.optimal_weights = np.zeros(n_models)
            self.optimal_weights[best_single_idx] = 1.0

        return self.optimal_weights

    def blend_predictions(self, test_predictions):
        """Blend test predictions using optimal weights."""
        print("\nBlending test predictions...")

        blended_log = np.zeros_like(test_predictions[self.model_names[0]])
        for name, w in zip(self.model_names, self.optimal_weights):
            blended_log += w * test_predictions[name]

        return blended_log


In [48]:

class PredictionCalibrator:
    """
    Post-processing calibration to directly optimize the competition metric.

    Now supports per survey calibration: learns separate (scale, shift) parameters
    for each survey, which can better capture survey-specific characteristics.
    """

    def __init__(self, config, metric):
        self.config = config
        self.metric = metric
        self.global_params = None
        self.per_survey_params = {}
        self.best_score = None
        self.score_before = None
        self.use_per_survey = True

    def _optimize_params(self, y_pred_orig, y_true_orig, survey_ids=None):
        """Find optimal (scale, shift) for given predictions."""

        def objective(params):
            scale, shift = params
            y_calibrated = scale * y_pred_orig + shift
            y_calibrated = np.maximum(y_calibrated, 0.01)
            return self.metric.calculate_from_original_scale(
                y_calibrated, y_true_orig, survey_ids
            )

        best_result = None
        best_score = float('inf')

        starting_points = [
            [1.0, 0.0],
            [1.0, 0.5],
            [1.0, -0.5],
            [1.05, 0.0],
            [0.95, 0.0],
            [1.02, 0.2],
            [0.98, -0.2],
            [1.1, 0.0],
            [0.9, 0.0],
            [1.0, 1.0],
            [1.0, -1.0],
            [1.1, -0.5],
            [0.9, 0.5],
            [1.15, -1.0],
            [0.85, 1.0],
        ]

        for start in starting_points:
            result = minimize(
                objective,
                start,
                method='Nelder-Mead',
                options={'maxiter': 2000, 'xatol': 1e-8, 'fatol': 1e-8}
            )

            if result.fun < best_score:
                best_score = result.fun
                best_result = result

        # try bounded optimization
        result_bounded = minimize(
            objective,
            [1.0, 0.0],
            method='L-BFGS-B',
            bounds=[(0.5, 1.5), (-5.0, 5.0)],
            options={'maxiter': 2000}
        )

        if result_bounded.fun < best_score:
            best_score = result_bounded.fun
            best_result = result_bounded

        return best_result.x, best_score

    def calibrate(self, y_pred_log, y_true_log, survey_ids):
        """
        Find optimal calibration parameters using per survey calibration.

        Learns separate (scale, shift) for each survey in the validation set.
        """
        print("\n" + "=" * 60)
        print("CALIBRATING PREDICTIONS (Per survey)")
        print("=" * 60)

        # back to original scale
        y_pred_orig = np.expm1(y_pred_log)
        y_true_orig = np.expm1(y_true_log)

        # scoring before calibration
        self.score_before = self.metric.calculate_from_original_scale(
            y_pred_orig, y_true_orig, survey_ids
        )
        print(f"\nScore Before calibration: {self.score_before*100:.2f}%")

        # global calibration
        print("\n--- Global calibration ---")
        self.global_params, global_score = self._optimize_params(
            y_pred_orig, y_true_orig, survey_ids
        )
        print(f"Global params: scale={self.global_params[0]:.4f}, shift={self.global_params[1]:.4f}")
        print(f"Global calibration score: {global_score*100:.2f}%")

        # per survey calibration
        print("\n--- Per survey calibration ---")
        unique_surveys = np.unique(survey_ids)

        y_calibrated_per_survey = np.zeros_like(y_pred_orig)

        for survey_id in unique_surveys:
            mask = survey_ids == survey_id
            y_pred_survey = y_pred_orig[mask]
            y_true_survey = y_true_orig[mask]

            # optimization for this survey
            params, score = self._optimize_params(y_pred_survey, y_true_survey, None)
            self.per_survey_params[survey_id] = params

            # applying calibration
            scale, shift = params
            y_calibrated_per_survey[mask] = scale * y_pred_survey + shift

            print(f"  Survey {survey_id}: scale={scale:.4f}, shift={shift:.4f}, score={score*100:.2f}%")

        # ensuring no negative values
        y_calibrated_per_survey = np.maximum(y_calibrated_per_survey, 0.01)

        # calculating per survey calibration score
        per_survey_score = self.metric.calculate_from_original_scale(
            y_calibrated_per_survey, y_true_orig, survey_ids
        )

        print(f"\nPer survey calibration score: {per_survey_score*100:.2f}%")

        # deciding which one to use
        if per_survey_score < global_score:
            print(f"\n✓ Using per survey calibration (better by {(global_score - per_survey_score)*100:.2f}%)")
            self.use_per_survey = True
            self.best_score = per_survey_score
            y_calibrated_orig = y_calibrated_per_survey
        else:
            print(f"\n✓ Using global calibration (better by {(per_survey_score - global_score)*100:.2f}%)")
            self.use_per_survey = False
            self.best_score = global_score
            scale, shift = self.global_params
            y_calibrated_orig = scale * y_pred_orig + shift
            y_calibrated_orig = np.maximum(y_calibrated_orig, 0.01)

        improvement = self.score_before - self.best_score
        print(f"\nFinal improvement: {improvement*100:.2f} percentage points")
        print(f"Score: {self.score_before*100:.2f}% → {self.best_score*100:.2f}%")

        # Convert back to log scale
        y_calibrated_log = np.log1p(y_calibrated_orig)

        return y_calibrated_log

    def apply_calibration(self, y_pred_log, test_survey_ids):
        """
        Apply the learned calibration parameters to test predictions.

        For per survey calibration, maps training surveys to test surveys:
        - Training: 100000, 200000, 300000
        - Test: 400000, 500000, 600000

        Mapping strategy: Use average of all training survey parameters,
        or map by survey order (100000->400000, 200000->500000, 300000->600000)
        """
        if self.global_params is None:
            raise ValueError("Must call calibrate() first to learn parameters")

        y_pred_orig = np.expm1(y_pred_log)

        if not self.use_per_survey:
            # using global calibration
            scale, shift = self.global_params
            y_calibrated_orig = scale * y_pred_orig + shift
        else:
            # using per survey calibration
            # Map training surveys to test surveys
            survey_mapping = {
                400000: 100000,  # Test survey 400000 uses params from train survey 100000
                500000: 200000,  # Test survey 500000 uses params from train survey 200000
                600000: 300000,  # Test survey 600000 uses params from train survey 300000
            }

            y_calibrated_orig = np.zeros_like(y_pred_orig)

            for test_sid in [400000, 500000, 600000]:
                mask = test_survey_ids == test_sid
                if mask.sum() == 0:
                    continue

                train_sid = survey_mapping[test_sid]

                if train_sid in self.per_survey_params:
                    scale, shift = self.per_survey_params[train_sid]
                else:
                    # falling back to global params
                    scale, shift = self.global_params

                y_calibrated_orig[mask] = scale * y_pred_orig[mask] + shift

            print(f"\nApplied per survey calibration:")
            for test_sid, train_sid in survey_mapping.items():
                if train_sid in self.per_survey_params:
                    scale, shift = self.per_survey_params[train_sid]
                    print(f"  Survey {test_sid} (using {train_sid} params): scale={scale:.4f}, shift={shift:.4f}")

        y_calibrated_orig = np.maximum(y_calibrated_orig, 0.01)
        y_calibrated_log = np.log1p(y_calibrated_orig)

        return y_calibrated_log


In [49]:

class SubmissionGenerator:
    """Generates submission files for DrivenData competition."""

    def __init__(self, config):
        self.config = config

    def generate(self, test_features, y_test_pred):
        """Generate submission files."""
        print("\n" + "=" * 60)
        print("GENERATING SUBMISSION FILES")
        print("=" * 60)

        submission_hh = pd.DataFrame({
            'survey_id': test_features['survey_id'],
            'hhid': test_features['hhid'],
            'cons_ppp17': y_test_pred
        })

        poverty_rates = []
        for survey_id in self.config.TEST_SURVEY_IDS:
            survey_preds = submission_hh[submission_hh['survey_id'] == survey_id]['cons_ppp17']
            row = {'survey_id': survey_id}
            for threshold in self.config.POVERTY_THRESHOLDS:
                col_name = f'pct_hh_below_{threshold:.2f}'
                pct_below = (survey_preds < threshold).mean()
                row[col_name] = pct_below
            poverty_rates.append(row)

        submission_rates = pd.DataFrame(poverty_rates)

        submission_hh.to_csv('predicted_household_consumption.csv', index=False)
        submission_rates.to_csv('predicted_poverty_distribution.csv', index=False)

        with zipfile.ZipFile('submission.zip', 'w') as zipf:
            zipf.write('predicted_household_consumption.csv')
            zipf.write('predicted_poverty_distribution.csv')

        print("\nFiles created:")
        print(f"  1. predicted_household_consumption.csv - Shape: {submission_hh.shape}")
        print(f"  2. predicted_poverty_distribution.csv - Shape: {submission_rates.shape}")
        print(f"  3. submission.zip")

        print(f"\nPrediction Statistics:")
        print(f"  Mean:   ${y_test_pred.mean():.2f}/day")
        print(f"  Median: ${np.median(y_test_pred):.2f}/day")
        print(f"  Min:    ${y_test_pred.min():.2f}/day")
        print(f"  Max:    ${y_test_pred.max():.2f}/day")

        print(f"\nPredicted Poverty Rates (at $5.26/day threshold):")
        for _, row in submission_rates.iterrows():
            print(f"  Survey {int(row['survey_id'])}: {row['pct_hh_below_5.26']*100:.1f}%")

        print("\n✓ Ready to submit to DrivenData!")

        return submission_hh, submission_rates

    def download(self):
        """Download the submission zip file."""
        from google.colab import files
        files.download('submission.zip')
        print("Download started!")


In [50]:

def main():
    """
    Main pipeline:
    1. Process data
    2. Train all models (XGBoost, LightGBM, CatBoost, Neural Network)
    3. Optimize ensemble weights
    4. Calibrate predictions with per survey calibration
    5. Generate submission
    """
    print("=" * 60)
    print("POVERTY PREDICTION PIPELINE")
    print("With Ensemble + Per survey Calibration")
    print("=" * 60)

    # Step 1: Initializing configuration
    print("\n[Step 1/7] Initializing configuration...")
    config = Config()

    # Step 2: Processing data
    print("\n[Step 2/7] Processing data...")
    processor = DataProcessor(config)
    data = processor.process_all()

    # Step 3: Initializing metric calculator
    print("\n[Step 3/7] Initializing competition metric...")
    metric = CompetitionMetric(config)

    # Step 4: Training all models
    print("\n[Step 4/7] Training models...")
    trainer = ModelTrainer(config, metric)
    models, val_predictions = trainer.train_all(data)

    # Step 5: Optimizing ensemble weights
    print("\n[Step 5/7] Optimizing ensemble...")
    optimizer = EnsembleOptimizer(config, metric)
    optimal_weights = optimizer.optimize_weights(
        val_predictions, data['y_val'], data['sid_val']
    )

    # getting blended validation predictions
    blended_val_pred = np.zeros_like(val_predictions['XGBoost'])
    for name, w in zip(optimizer.model_names, optimal_weights):
        blended_val_pred += w * val_predictions[name]

    # Step 6: Calibrating predictions (per survey)
    print("\n[Step 6/7] Calibrating predictions (per survey)...")
    calibrator = PredictionCalibrator(config, metric)
    calibrated_val_pred = calibrator.calibrate(
        blended_val_pred,
        data['y_val'].values,
        data['sid_val']
    )

    # generating test predictions
    test_predictions = trainer.generate_test_predictions(data['X_test'], data['X_test_scaled'])

    # blending test predictions
    blended_test_pred = optimizer.blend_predictions(test_predictions)

    # getting test survey IDs for per survey calibration
    test_survey_ids = data['test']['survey_id'].values

    # applying calibration to test predictions (with survey IDs)
    calibrated_test_pred = calibrator.apply_calibration(blended_test_pred, test_survey_ids)

    # converting to original scale
    y_test_pred = np.expm1(calibrated_test_pred)

    print(f"\nFinal Test Predictions (after per survey calibration):")
    print(f"  Mean:   ${y_test_pred.mean():.2f}/day")
    print(f"  Median: ${np.median(y_test_pred):.2f}/day")
    print(f"  Min:    ${y_test_pred.min():.2f}/day")
    print(f"  Max:    ${y_test_pred.max():.2f}/day")

    # per survey statistics
    print(f"\nPer survey prediction statistics:")
    for sid in [400000, 500000, 600000]:
        mask = test_survey_ids == sid
        print(f"  Survey {sid}: Mean=${y_test_pred[mask].mean():.2f}, Median=${np.median(y_test_pred[mask]):.2f}")

    # Step 7: Generating submission
    print("\n[Step 7/7] Generating submission...")
    submission = SubmissionGenerator(config)
    submission_hh, submission_rates = submission.generate(data['test'], y_test_pred)

    print("\n" + "=" * 60)
    print("PIPELINE COMPLETE!")
    print("=" * 60)

    # Final Summary
    print("\n" + "=" * 60)
    print("FINAL SUMMARY")
    print("=" * 60)
    print(f"Best single model: {optimizer.best_single_model} ({optimizer.best_single_score*100:.2f}%)")
    print(f"Ensemble weights: ", end="")
    for name, w in zip(optimizer.model_names, optimal_weights):
        if w > 0.01:
            print(f"{name}={w*100:.1f}% ", end="")
    print()

    if calibrator.use_per_survey:
        print(f"Calibration type: Per survey")
        for train_sid, params in calibrator.per_survey_params.items():
            print(f"  Survey {train_sid}: scale={params[0]:.4f}, shift={params[1]:.4f}")
    else:
        print(f"Calibration type: GLOBAL")
        print(f"  scale={calibrator.global_params[0]:.4f}, shift={calibrator.global_params[1]:.4f}")

    print(f"\nScore before calibration: {calibrator.score_before*100:.2f}%")
    print(f"Score after calibration: {calibrator.best_score*100:.2f}%")

    return {
        'config': config,
        'data': data,
        'metric': metric,
        'trainer': trainer,
        'optimizer': optimizer,
        'calibrator': calibrator,
        'submission': submission,
        'y_test_pred': y_test_pred
    }


In [51]:

results = main()
results['submission'].download()

POVERTY PREDICTION PIPELINE
With Ensemble + Per survey Calibration

[Step 1/7] Initializing configuration...

[Step 2/7] Processing data...
STARTING DATA PROCESSING PIPELINE
Loading data...
Training data shape: (104234, 89)
Test data shape: (103023, 88)
Handling missing values...
Missing values after filling: 0
Encoding binary columns...
Binary columns encoded: 59
Encoding multiclass columns...
Multiclass columns encoded: 5
Defining feature columns...
Total feature columns: 84
Preparing feature matrices...
X shape: (104234, 84)
y shape: (104234,)
X_test shape: (103023, 84)
Creating train/validation splits...
Training set: 83387 samples
Validation set: 20847 samples
Surveys in validation: [100000 200000 300000]
Scaling features...

DATA PROCESSING COMPLETE

[Step 3/7] Initializing competition metric...

[Step 4/7] Training models...

TRAINING ALL MODELS

Training XGBoost...
Training time: 7.54 seconds
Validation Score: 8.70% (Poverty: 7.08%, Consumption: 23.28%)

Training LightGBM...
Tr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download started!
